In [165]:
import pandas as pd
import math
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

m019 = pd.read_csv('M019_08-30-24_Day4_CHIMERADLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.csv')
m020 = pd.read_csv('M020_08-30-24_Day4_CHIMERADLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.csv')
m021 = pd.read_csv('M021_09-01-24_Day2_CHIMERADLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.csv')
m022 = pd.read_csv('M022_09-03-24_Day4_CHIMERA-VideoDLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.csv')

In [166]:
m019.head() # possible x-coord columns: 1, 4, 7, 10

,scorer,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.1,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.2,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.3,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.4,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.5,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.6,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.7,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.8,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.9,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.10,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.11
0,bodyparts,eye,eye,eye,ear,ear,ear,shoulder,shoulder,shoulder,objectA,objectA,objectA
1,coords,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood
2,0,424.29688,423.8478,0.6533753,447.07108,429.03516,0.5795554,483.73245,378.73856,0.69176674,399.02893,404.08994,0.81403875
3,1,421.59845,426.11047,0.6459025,446.66553,429.2485,0.58834827,484.0062,378.74176,0.674235,400.58398,405.39468,0.8113508
4,2,424.42755,423.69998,0.65993315,447.72086,429.06137,0.61605036,484.65308,379.53146,0.6882895,399.4229,404.02182,0.81882113


In [167]:
def average_value(data, colname):
    value = float(data[data[colname] != 0][colname].mean())
    print("Average " + colname + " (pixels per second): " + str(value))
    return value

In [168]:
def linear_velocity(data, body_part_col_num):
    # data: dataset to use
    # body_part_col_num: the column # of the x coordinate column to use
    
    # velocity: ((x2-x1)^2 + (y2-y1)^2)^0.5 / ((t2-t1) / 10000)
    data['linear_velocity'] = 0
    
    for i in range(1+int(time), len(data), int(time)):  # loop through every time-th frame
        # data starts in row 2
        x2 = float(data.iat[i, body_part_col_num])
        y2 = float(data.iat[i, body_part_col_num+1])
        # previous frame is 5 frames back, subtract one less than that to include that frame
        x1 = float(data.iat[i-(int(time)-1), body_part_col_num])
        y1 = float(data.iat[i-(int(time)-1), body_part_col_num+1])

        data.at[i, 'linear_velocity'] = float((((x2 - x1)**2 + (y2 - y1)**2)**0.5) / time)
    
    return data

In [169]:
def linear_acceleration(data):
    # ((v2 - v1) / time)
    # how it works: looking through for velocity values calculated and giving an acceleration for each pair of velocity values
    data['linear_acceleration'] = 0

    previous_velocity = None
    
    for i in range(2, len(data), 1):
        current_velocity = float(data.loc[i, 'linear_velocity'])
        if current_velocity != 0: # if there is a previous velocity, calculate acceleration
            if previous_velocity is not None:
                acceleration = float((current_velocity - previous_velocity) / time)
                
                # Fill the acceleration value for the current index
                data.at[i, 'linear_acceleration'] = acceleration
            
            # Update previous velocity and index
            previous_velocity = current_velocity
    return data

In [170]:
def angular_velocity(data, body_part_col_num):
    # data: dataset to use
    # body_part_col_num: the column # of the x coordinate column to use
    
    # angular velocity: (arctan((y2-y1)/(x2-x1))*180) / pi) / time
    data['angular_velocity'] = 0
    
    for i in range(1+int(time), len(data), int(time)):  # loop through every time-th frame
        # data starts in row 2
        x2 = float(data.iat[i, body_part_col_num])
        y2 = float(data.iat[i, body_part_col_num+1])
        # previous frame is 5 frames back, subtract one less than that to include that frame
        x1 = float(data.iat[i-(int(time)-1), body_part_col_num])
        y1 = float(data.iat[i-(int(time)-1), body_part_col_num+1])

        # in radians !! use math.degrees(math.atan(value)) if degrees wanted
        data.at[i, 'angular_velocity'] = (math.atan((y2-y1) / (x2-x1))) / time
    
    return data

In [171]:
def angular_acceleration(data):
     # ((v2 - v1) / time)
    # how it works: looking through for velocity values calculated and giving an acceleration for each pair of velocity values
    data['angular_acceleration'] = 0

    previous_angular_velocity = None
    
    for i in range(2, len(data), 1):
        current_angular_velocity = float(data.loc[i, 'angular_velocity'])
        if current_angular_velocity != 0: # if there is a previous velocity, calculate acceleration
            if previous_angular_velocity is not None:
                angular_accel = float((current_angular_velocity - previous_angular_velocity) / time)
                
                # Fill the acceleration value for the current index
                data.at[i, 'angular_acceleration'] = angular_accel
            
            # Update previous velocity and index
            previous_angular_velocity = current_angular_velocity
    return data

Run code below after initializing all these functions:

In [172]:
# typically column_num_body_part = 1 works but m020 is giving issues.
# change variables depending on what you want !!
# 1 frame / second
time = 5

def compute(dataset, column_num_body_part):
    linear_velocity(dataset, column_num_body_part) # currently, using eye (x is column 1)
    linear_acceleration(dataset) # can't do accel values without doing velocity first
    angular_velocity(dataset, column_num_body_part)
    angular_acceleration(dataset)
    
    # Average values
    average_value(dataset, 'linear_velocity')
    average_value(dataset, 'linear_acceleration')
    average_value(dataset, 'angular_velocity')
    average_value(dataset, 'angular_acceleration')
    print(" ")
    
    return dataset

In [173]:
print("Average velocity, acceleration values for m019")
compute(m019, 1)

# print("Average velocity, acceleration values for m020")
# compute(m020, 1)

print("Average velocity, acceleration values for m021")
compute(m021, 1)

print("Average velocity, acceleration values for m022")
compute(m022, 1)

Average velocity, acceleration values for m019
Average linear_velocity (pixels per second): 0.43847454588238177
Average linear_acceleration (pixels per second): 3.5796803718439415e-06
Average angular_velocity (pixels per second): 0.0071429872091130974
Average angular_acceleration (pixels per second): 7.532440309179947e-06
 
Average velocity, acceleration values for m021
Average linear_velocity (pixels per second): 0.5209485362885496
Average linear_acceleration (pixels per second): -1.447266699306411e-05
Average angular_velocity (pixels per second): -0.01563348700913233
Average angular_acceleration (pixels per second): -3.942203884734157e-05
 
Average velocity, acceleration values for m022
Average linear_velocity (pixels per second): 0.5048547369123025
Average linear_acceleration (pixels per second): -3.7726323890104974e-05
Average angular_velocity (pixels per second): 0.0026637714264173148
Average angular_acceleration (pixels per second): 2.6607278047335096e-05
 


,scorer,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.1,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.2,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.3,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.4,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.5,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.6,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.7,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.8,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.9,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.10,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.11,linear_velocity,linear_acceleration,angular_velocity,angular_acceleration
0,bodyparts,eye,eye,eye,ear,ear,ear,shoulder,shoulder,shoulder,objectA,objectA,objectA,0.000000,0.000000,0.000000,0.000000
1,coords,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,0.000000,0.000000,0.000000,0.000000
2,0,441.16534,360.99457,0.63528967,459.9667,371.0939,0.71293545,492.5,319.4064,0.8074962,420.1927,341.1103,0.72478575,0.000000,0.000000,0.000000,0.000000
3,1,440.48157,361.81207,0.64748126,459.76987,371.92917,0.7048725,492.3438,319.2242,0.8102001,419.70264,341.24332,0.7409266,0.000000,0.000000,0.000000,0.000000
4,2,440.17163,360.89597,0.6297608,459.75397,371.2705,0.7021736,492.18436,319.15866,0.8166534,420.0778,340.93524,0.7328238,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3398,3396,440.44653,356.733,0.7987046,462.39203,363.90952,0.7932677,474.53497,317.4811,0.7396126,430.75165,345.57285,0.82637274,0.000000,0.000000,0.000000,0.000000
3399,3397,440.74576,356.582,0.7719606,463.01773,363.851,0.800965,474.6891,317.19415,0.7347301,431.32492,345.34167,0.78673697,0.000000,0.000000,0.000000,0.000000
3400,3398,440.67255,356.6742,0.761404,462.92603,363.88718,0.80432796,474.76804,317.20694,0.7394566,431.24713,345.3926,0.7731357,0.000000,0.000000,0.000000,0.000000
3401,3399,440.58694,356.51453,0.7741602,463.15207,363.952,0.8039773,474.61234,317.0517,0.7364364,431.29514,345.32455,0.7860209,0.028557,-0.012228,0.081015,0.010015


In [174]:
m019

,scorer,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.1,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.2,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.3,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.4,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.5,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.6,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.7,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.8,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.9,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.10,DLC_Resnet50_chimera_launch_videosSep15shuffle1_snapshot_200.11,linear_velocity,linear_acceleration,angular_velocity,angular_acceleration
0,bodyparts,eye,eye,eye,ear,ear,ear,shoulder,shoulder,shoulder,objectA,objectA,objectA,0.000000,0.000000,0.000000,0.000000
1,coords,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,0.000000,0.000000,0.000000,0.000000
2,0,424.29688,423.8478,0.6533753,447.07108,429.03516,0.5795554,483.73245,378.73856,0.69176674,399.02893,404.08994,0.81403875,0.000000,0.000000,0.000000,0.000000
3,1,421.59845,426.11047,0.6459025,446.66553,429.2485,0.58834827,484.0062,378.74176,0.674235,400.58398,405.39468,0.8113508,0.000000,0.000000,0.000000,0.000000
4,2,424.42755,423.69998,0.65993315,447.72086,429.06137,0.61605036,484.65308,379.53146,0.6882895,399.4229,404.02182,0.81882113,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3398,3396,420.00443,414.33423,0.82909304,437.364,425.3304,0.80284613,464.076,396.21606,0.42292204,403.99683,401.1612,0.9275156,0.000000,0.000000,0.000000,0.000000
3399,3397,420.24695,414.38205,0.8279493,437.71136,425.08295,0.80639267,466.60312,393.3155,0.42340115,403.6406,401.0514,0.9249245,0.000000,0.000000,0.000000,0.000000
3400,3398,419.76544,414.6461,0.8290252,437.48093,425.37357,0.7987297,465.77542,394.15564,0.44217536,403.6429,401.37405,0.92157567,0.000000,0.000000,0.000000,0.000000
3401,3399,420.13016,414.47696,0.83550334,437.941,425.09436,0.79302955,463.6807,395.91956,0.41949165,403.4845,400.95303,0.93598354,0.117406,0.010691,-0.071398,-0.014791
